**Install Dependencies**

In [0]:
!pip3 install tensorflow==1.10.1 keras==2.2.2 tensorflowjs==0.6.0

In [0]:
import traceback
import logging
import tensorflow as tf
import keras.backend as K

from pathlib import Path

from google.colab import files

from keras import Model, Input
from keras.applications import MobileNet
from keras.engine.saving import load_model

from tensorflow.python.framework import graph_util, graph_io
from tensorflowjs.converters import load_keras_model

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

***Cleanup any existing models if necessary***

In [0]:
!!rm -rf *.h5 *.tflite *.json *.bin

**Upload your Tensorflow.js Artifacts Here**

i.e., The weights manifest **model.json** and the binary weights file **model-weights.bin**

In [0]:
files.upload()

**Export Configuration**

In [0]:
#@title Export Configuration

# TensorFlow.js arguments

config_json = "model.json" #@param {type:"string"}
weights_path_prefix = "." #@param {type:"string"}
load_weights = True #@param {type:"boolean"}
use_unique_name_scope = True #@param {type:"boolean"}

# MobileNet model options

input_node_name = "the_input" #@param {type:"string"}
alpha = 0.25 #@param {type:"slider", min:0.25, max:1, step:0.25}
image_size = 224 #@param ["128", "160", "192", "224"] {type:"raw"}
intermediate_node_name = "conv_pw_13_relu" #@param {type:"string"}

# Frozen Graph options
output_node_prefix = "output_node" #@param {type:"string"}

# Path to exported models
keras_model_file= "model.h5" #@param {type:"string"}
frozen_model_file = "model.pb" #@param {type:"string"}
tflite_model_file = "model.tflite" #@param {type:"string"}



**Model Converter**

The following class converts a TensorFlow.js model to a TFLite FlatBuffer

In [0]:
class ModelConverter:
    """
    Creates a ModelConverter class from a TensorFlow.js model file.

    Args:
      config_json_path: Full filepath of weights manifest file containing the model architecture.
      weights_path_prefix: Full filepath to the directory in which the weights binaries exist.
      use_unique_name_scope: Use a unique ID as the name scope for the loaded model.
      tflite_model_file: Name of the TFLite FlatBuffer file to be exported.

    Returns:
      ModelConverter class.
    """

    def __init__(self,
                 config_json_path,
                 weights_path_prefix,
                 use_unique_name_scope,
                 tflite_model_file
                 ):
        self.config_json_path = config_json_path
        self.weights_path_prefix = weights_path_prefix
        self.use_unique_name_scope = use_unique_name_scope

        self.input_node_name = None
        self.intermediate_node_name = None
        self.output_node_name = None

        self.tflite_model_file = tflite_model_file

    def convert(self, keras_model_file, output_node_prefix, frozen_model_file):
        frozen_model_file = self._freeze_graph_from_keras(keras_model_file,
                                                          output_node_prefix,
                                                          frozen_model_file=frozen_model_file,
                                                          num_outputs=1)

        self._deserialize_tflite_from_frozen_graph(frozen_model_file,
                                                   self.input_node_name,
                                                   self.output_node_name,
                                                   self.tflite_model_file)

        logger.info('The TFLite model has been generated')

    def save_keras_model(self, keras_model_file, **kwargs):
        """
        Converts a Keras model to a frozen TensorFlow Protocol Buffer.

        Args:
            keras_model_file: Filename of the Keras H5 model to be saved
            input_node_name: Name of the input layer for the model being built
            intermediate_node_name: Name of the intermediate Depthwise Convolutional Block in the base model (MobileNet)
            image_size: Size of the input image
            alpha: Width of the MobileNet network
        """
        top_model = load_keras_model(self.config_json_path, self.weights_path_prefix,
                                     weights_data_buffers=None,
                                     load_weights=True,
                                     use_unique_name_scope=self.use_unique_name_scope)
        base_model = self.build_base_model(**kwargs)
        merged_model = self.merge(base_model, top_model)
        merged_model.save(keras_model_file)
        logger.info("Keras HDF5 saved to {}".format(keras_model_file))

    def merge(self, base_model, top_model):
        logger.info("Creating model...")
        layer = base_model.get_layer(self.intermediate_node_name)
        model = Model(inputs=base_model.input, outputs=top_model(layer.output))
        return model

    def build_base_model(self,
                         depth_multiplier=1,
                         **kwargs):
        self.input_node_name = kwargs.get('input_node_name')
        self.intermediate_node_name = kwargs.get('intermediate_node_name')

        image_size = kwargs.get('image_size')
        alpha = kwargs.get('alpha')
        input_shape = (image_size, image_size, 3)

        input_tensor = Input(shape=input_shape, name=self.input_node_name)
        base_model = MobileNet(input_shape=input_shape,
                               alpha=alpha,
                               depth_multiplier=depth_multiplier,
                               input_tensor=input_tensor,
                               include_top=False)
        return base_model

    def _freeze_graph_from_keras(self, model_file,
                                 output_node_prefix,
                                 output_folder='.',
                                 frozen_model_file="model.pb",
                                 num_outputs=1
                                 ):
        K.clear_session()
        K.set_learning_phase(0)
        K.set_image_data_format('channels_last')

        model = load_model(model_file, compile=False)
        sess = K.get_session()

        pred = [None] * num_outputs
        pred_node_names = [None] * num_outputs

        for i in range(num_outputs):
            pred_node_names[i] = output_node_prefix + str(i)
            pred[i] = tf.identity(model.outputs[i], name=pred_node_names[i])

        logger.info('Output nodes names are: {}'.format(pred_node_names))

        self.output_node_name = pred_node_names[0]

        constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
        graph_io.write_graph(constant_graph, '.', frozen_model_file, as_text=False)
        logger.info('Saved the frozen graph (ready for inference) at: {}'
                     .format(str(Path(output_folder) / frozen_model_file)))

        return frozen_model_file

    @staticmethod
    def _deserialize_tflite_from_frozen_graph(frozen_model_file,
                                              input_node_name,
                                              output_node_name,
                                              tflite_model_file):
        K.clear_session()

        input_arrays = [input_node_name]
        output_arrays = [output_node_name]

        converter = tf.contrib.lite.TocoConverter.from_frozen_graph(
            frozen_model_file, input_arrays, output_arrays)
        tflite_model = converter.convert()

        with open(tflite_model_file, "wb") as file:
            file.write(tflite_model)


In [0]:
try:
    converter = ModelConverter(args.config_json_path,
                               args.weights_path_prefix,
                               args.use_unique_name_scope,
                               args.tflite_model_file)

    converter.save_keras_model(args.keras_model_file,
                               input_node_name=args.input_node_name,
                               intermediate_node_name=args.intermediate_node_name,
                               image_size=args.image_size,
                               alpha=args.alpha)

    converter.convert(args.keras_model_file, args.output_node_prefix, args.frozen_model_file)

except ValueError as e:
    print(traceback.format_exc())
    print("Error occurred while converting")

In [0]:
files.download(tflite_model_file)